<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2024notebooks/2024_0510PCA_tSNE_and_Logistic_regression_of_Olivetti_face.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# オリベッティ顔データベースを用いた顔認識

* Date: 2024_0510
* Author: 浅川 伸一 educ0233@komazawa-u.ac.jp

* 自身の Google Drive にコピーした上で実行してください。

In [ ]:
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
from sklearn.datasets import fetch_olivetti_faces
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib
    import japanize_matplotlib

data = fetch_olivetti_faces()
X, y = data.data, data.target

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())

In [ ]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces

data = fetch_olivetti_faces()
X, y = data.data, data.target

print(f"目標とするクラス(画像中の人物の数) :{np.unique(y)}")


In [ ]:
nrows = 40   # nrows 人分のデータを表示
nrows = 4
ncols = 10
fig, fig_axes = plt.subplots(ncols=ncols, nrows=nrows, figsize=(ncols * 1.4, nrows * 1.4), constrained_layout=True)
# constrained_layout は subplot や 凡例やカラーバーなどの装飾を自動的に調整して，
# ユーザが要求する論理的なレイアウトをできるだけ維持しながら， 図ウィンドウに収まるようにします。

for i in range(nrows):
    for j in range(ncols):
        x = i * 10 + j
        fig_axes[i][j].imshow(X[x].reshape(64,64), cmap='gray')
        fig_axes[i][j].axis('off')
        fig_axes[i][j].set_title(f'num:{x}, ID:{y[x]}')

# 機械学習手法による顔認識

## データの分割，訓練データとテストデータ

データを 2 分割して，訓練データセットとテストデータセットに分割します。
分割した訓練データセットでモデルのパラメータを学習し，しかる後に，テストデータセットで，その汎化性能を評価します。
このとき，テストデータセットでの性能が高いモデルが良いモデルということになります。

オリベッティ顔データセットには， 各被験者の 10 枚の顔画像が含まれています。
このうち，例えば 90% を訓練データとし，10% をテストデータとして使用することを考えます。
各顔データの訓練画像とテスト画像の数が同じになるように stratify 機能を使用してます。
したがって，各被験者には 9 枚の訓練用画像と 1 枚のテスト用画像が用意されることになります。
訓練データとテストデータの割合は split_ratio 変更することができます。


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import seaborn as sns  # ヒートマップ描画のために使用します

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.decomposition import PCA

# split_ratio = 0.1 としているので，訓練データ対テストデータが 9:1 になります
split_ratio = 0.3
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=split_ratio, stratify=y, random_state=42)
print(f'X_train 訓練画像のサイズ: {X_train.shape}')
print(f'y_train 教師信号データのサイズ: {y_train.shape}')

## ロジスティック回帰による予測

In [ ]:
params = {'max_iter':10 ** 3,
          'C':1e3,
          'penalty':'l2'}

model = LogisticRegression(**params)
model.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
y_hat = model.predict(X_test)  # テストデータを使って予測を行い結果を y_hat に格納
print(f"ロジスティック回帰を用いた分類精度: {metrics.accuracy_score(y_test, y_hat):.3f}")

# 混同行列の表示
plt.figure(figsize=(8,6))
sns.heatmap(metrics.confusion_matrix(y_test, y_hat))

In [ ]:
model_lda = LinearDiscriminantAnalysis()

model_lda.fit(X_train, y_train)    # 訓練データを用いて線形判別分析モデルを訓練
lda_y_hat = model_lda.predict(X_test)  # テストデータを使って予測を行い結果を lda_y_hat に格納
print(f"線形判別分析を用いた分類精度: {metrics.accuracy_score(y_test, lda_y_hat):.3f}")

# 混同行列の表示
plt.figure(figsize=(8,6))
sns.heatmap(metrics.confusion_matrix(y_test, lda_y_hat))

In [ ]:
model_pca = PCA(n_components=4, random_state=42)
model_pca.fit(X)
print(model_pca.explained_variance_ratio_, model_pca.explained_variance_ratio_.sum())
print(model_pca.singular_values_)
#help(model_pca)

In [ ]:

model_pca20 = PCA(n_components=20, random_state=42)
model_pca20.fit(X)
print(model_pca20.explained_variance_ratio_, model_pca20.explained_variance_ratio_.sum())
print(model_pca20.singular_values_)

In [ ]:
model_pca20.transform(X).shape

In [ ]:
#help(model_pca20.transform)
model_pca20.transform(X)[0]
print(X.shape)
#help(model_pca20.score)
#model_pca20.score(X)
print(model_pca.components_.shape)
print(X_proj.shape)

Wt20 = model_pca20.components_
#(W @ W.T).shape
print(Wt20.T.shape)
print((X @ Wt20.T).shape)
Wt20d = Wt20.T @ Wt20
Wd20_inv = np.linalg.inv(Wt20d)
#type(np.invert)
XWt20 = X @ Wt20.T
P20_ = X @ Wd20_inv


In [ ]:
print(W.T.shape)
print(P_.shape)
plt.imshow(P20_[0].reshape(64,64), cmap='gray')

## PCA の結果をプロット

In [ ]:
figsize=(8,8)
dotsize=120
#dotsize=30
fontsize=10
fontsize=12
#fontsize=7
dotcolor=None
dim1, dim2 = 0, 1

model_pca.fit(X)
X_proj = model_pca.transform(X)

plt.figure(figsize=figsize)
for i, (vec) in enumerate(X_proj):
    _x, _y = vec[dim1], vec[dim2]
    plt.annotate(str(y[i]), (_x, _y), ha='center', fontsize=fontsize)

plt.scatter(X_proj[:,dim1], X_proj[:,dim2], marker='.', s=dotsize, c=y)
plt.show()


## tSNE によるプロット

In [ ]:
from sklearn import manifold

_tsne = manifold.TSNE(n_components=2, perplexity=40, random_state=42).fit_transform(X)

figsize=(8,8)
#figsize=(12,12)
dotsize=120
fontsize=12
dotcolor=None
dim1, dim2 = 0, 1

plt.figure(figsize=figsize)
for i, (vec) in enumerate(_tsne):
    _x, _y = vec[dim1], vec[dim2]
    plt.annotate(str(y[i]), (_x, _y), ha='center', fontsize=fontsize)

plt.scatter(_tsne[:,dim1], _tsne[:,dim2], marker='.', s=dotsize, c=y)
plt.show()


# 固有顔を特徴として用いた分類 サポートベクターマシンによる

In [ ]:
n_components = 150
print(f"総数 {X_train.shape[0]} 枚の顔画像から，上位 {n_components} 成分の固有顔 eigenfaces を抽出")

pca = PCA(n_components=n_components, svd_solver="randomized", whiten=True).fit(X_train)

height, width = 64, 64
eigenfaces = pca.components_.reshape((n_components, height, width))

print("入力データを固有顔空間の直交基底に射影")
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

print("分類器を訓練データセットへ適合")
param_grid = {
    "C": loguniform(1e3, 1e5),
    "gamma": loguniform(1e-4, 1e-1),
}
clf = RandomizedSearchCV(
    SVC(kernel="rbf", class_weight="balanced"), param_grid, n_iter=10
)
clf = clf.fit(X_train_pca, y_train)
print("グリッドサーチによる最良分類推定値")
print(clf.best_estimator_)

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay

print("Predicting people's names on the test set")
y_pred = clf.predict(X_test_pca)

target_names = np.array([str(x) for x in list(set(y_test))])
print(classification_report(y_test, y_pred, target_names=target_names))

fig, ax = plt.subplots(figsize=(10,10))
cmp = ConfusionMatrixDisplay.from_estimator(
    clf, X_test_pca, y_test, display_labels=target_names,
)

cmp.plot(ax=ax)